In [4]:
import pandas as pd
from pyspark.sql import *

In [10]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
viajes = SQLContext(sc).read.csv("viajes.csv").rdd
viajes = viajes.map(lambda x:(pd.to_numeric(x[0],"coerce"),pd.to_numeric(x[1],"coerce"),\
                              pd.to_numeric(x[2],"coerce"),pd.to_numeric(x[3],"coerce"),\
                              pd.to_datetime(x[4],"coerce"),pd.to_numeric(x[5],"coerce"))).cache()

In [15]:
#UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus vehículos. 
#Existe un proceso que nos devuelve un RDD llamado trip_summary con los siguientes campos: 
#(driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled), Programar usando 
#Py Spark un programa que nos indique cual fue el conductor con mayor promedio de distancia recorrida 
#por viaje para Abril de 2016.

#RDD: (driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled)
abril_2019 = viajes.filter(lambda x: (x[4].year==2019)and(x[4].month==4))#Lo cambio a 2019 por el csv
promedio_recorrido = abril_2019.map(lambda x:(x[0],(x[5],1))).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])).\
        map(lambda x: (x[0],x[1][0]/x[1][1]))#(driver,promedio_distancia)
promedio_recorrido.reduce(lambda x,y: x if x[1]>y[1] else y)

(17, 920.0)